In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score
import scipy.stats as stats
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from datetime import datetime
from category_encoders import OrdinalEncoder, TargetEncoder
from catboost import CatBoostClassifier, CatBoostRegressor
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Functions

def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    roc = roc_auc_score(y_test, predictions)
    print('Model Performance')
    print('Average Error: {:0.4f} degrees'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%'.format(accuracy))
    print(f'AUC = {roc}')
    return accuracy

In [3]:
train = pd.read_csv('../Data/training_set_features.csv', index_col='respondent_id')
test = pd.read_csv('../Data/test_set_features.csv', index_col ='respondent_id')
labels = pd.read_csv('../Data/training_set_labels.csv', index_col='respondent_id')

In [4]:
train.loc[(train['age_group'] == '65+ Years') & (train['employment_status'].isnull()), 'employment_status'] = 'Not in Labor Force'

In [5]:
num_cols = list(train.select_dtypes('number').columns)

cat_cols = [
    'race',
    'sex',
    'marital_status',
    'rent_or_own',
    'hhs_geo_region',
    'census_msa',
    'employment_industry',
    'employment_occupation'
]

ord_cols = [
    'age_group',
    'education',
    'income_poverty',
    'employment_status'
]




#Impute Train
for col in num_cols:
    train[col] = train[col].fillna(value=-1)
    test[col] = test[col].fillna(value=-1)

for col in (cat_cols + ord_cols):
    train[col] = train[col].fillna(value='None')
    test[col] = test[col].fillna(value='None')
test_labels = labels.copy()    

In [6]:
train['age_group'] = train['age_group'].map({
    '18 - 34 Years': 1,
    '35 - 44 Years': 2,
    '45 - 54 Years': 3,
    '55 - 64 Years': 4,
    '65+ Years': 5
})

train['education'] = train['education'].map({
    '< 12 Years': 1,
    '12 Years': 2,
    'Some College': 3,
    'College Graduate': 4,
    'None': -1
})

train['income_poverty'] = train['income_poverty'].map({
    'None': -1,
    'Below Poverty': 1,
    '<= $75,000, Above Poverty': 2,
    '> $75,000': 3
})

train['employment_status'] = train['employment_status'].map({
    'None': -1,
    'Unemployed': 1,
    'Employed': 2,
    'Not in Labor Force': 3
})




test['education'] = test['education'].map({
    '< 12 Years': 1,
    '12 Years': 2,
    'Some College': 3,
    'College Graduate': 4,
    'None': -1
})

test['income_poverty'] = test['income_poverty'].map({
    'None': -1,
    'Below Poverty': 1,
    '<= $75,000, Above Poverty': 2,
    '> $75,000': 3
})

test['employment_status'] = test['employment_status'].map({
    'None': -1,
    'Unemployed': 1,
    'Employed': 2,
    'Not in Labor Force': 3
})

In [7]:
for x in train[ord_cols].columns:
    print(x, train[x].unique())

age_group [4 2 1 5 3]
education [ 1  2  4  3 -1]
income_poverty [ 1  2  3 -1]
employment_status [ 3  2  1 -1]


In [8]:
all_cols = train.columns

train_test = train.copy()

In [9]:
h1n1_labels = labels[['h1n1_vaccine']]
seas_labels = labels[['seasonal_vaccine']]

# Transformation

In [10]:
cat_cols = train.select_dtypes('object').columns

In [11]:
h1n1_train = train.copy()
seas_train = train.copy()

h1n1_scaler = StandardScaler()
h1n1_train[num_cols] = h1n1_scaler.fit_transform(h1n1_train[num_cols])

seas_scaler = StandardScaler()
seas_train[num_cols] = seas_scaler.fit_transform(seas_train[num_cols])

h1n1_train_trans = h1n1_train
seas_train_trans = seas_train

In [12]:
categorical_features_indices = np.where(train.dtypes != float)[0]

# H1N1

## CatBoost and Optuna

In [13]:
X = h1n1_train_trans
y = h1n1_labels

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [15]:
from catboost import CatBoostClassifier
from catboost import Pool, cv
from sklearn.metrics import roc_curve, roc_auc_score
import optuna

In [16]:
train_dataset = Pool(data=X_train,
                    label=y_train,
                    cat_features=categorical_features_indices)

In [17]:
def objective(trial):
    param = {
        'iterations':trial.suggest_categorical('iterations', [100,200,300,500,1000,1200,1500,1700,2000]),
        'learning_rate':trial.suggest_float("learning_rate", 0.001, 0.3),
        'random_strength':trial.suggest_int("random_strength", 1,10),
        'bagging_temperature':trial.suggest_int("bagging_temperature", 0,10),
        'max_bin':trial.suggest_categorical('max_bin', [4,5,6,8,10,20,30]),
        'grow_policy':trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'min_data_in_leaf':trial.suggest_int("min_data_in_leaf", 1,10),
        'od_type' : "Iter",
        'od_wait' : 100,
        "depth": trial.suggest_int("max_depth", 2,10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
         'one_hot_max_size':trial.suggest_categorical('one_hot_max_size', [5,10,12,25,100,500,1024]),
        'custom_metric' : ['AUC'],
        "loss_function": "Logloss",
        'auto_class_weights':trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']),
        }

    scores = cv(train_dataset,
            param,
            fold_count=7, 
            early_stopping_rounds=8,         
            plot=False, verbose=False)

    return scores['test-AUC-mean'].max()

In [18]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=75)

[I 2021-07-12 19:19:43,008] A new study created in memory with name: no-name-0f64e8c9-31bf-4b43-802a-5340d9a69b62


Stopped by overfitting detector  (8 iterations wait)


[I 2021-07-12 19:19:54,512] Trial 0 finished with value: 0.8617267124560974 and parameters: {'iterations': 200, 'learning_rate': 0.2127137007610176, 'random_strength': 1, 'bagging_temperature': 10, 'max_bin': 4, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 3, 'max_depth': 5, 'l2_leaf_reg': 0.0003636636071695854, 'one_hot_max_size': 5, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 0 with value: 0.8617267124560974.


Stopped by overfitting detector  (8 iterations wait)


[I 2021-07-12 19:19:58,891] Trial 1 finished with value: 0.8657620717799731 and parameters: {'iterations': 100, 'learning_rate': 0.2728868002215558, 'random_strength': 3, 'bagging_temperature': 7, 'max_bin': 10, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 1, 'max_depth': 3, 'l2_leaf_reg': 2.833171661121252e-08, 'one_hot_max_size': 25, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 1 with value: 0.8657620717799731.
[I 2021-07-12 19:20:51,764] Trial 2 finished with value: 0.8641775823645944 and parameters: {'iterations': 200, 'learning_rate': 0.023139350868493017, 'random_strength': 4, 'bagging_temperature': 1, 'max_bin': 4, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 2, 'max_depth': 8, 'l2_leaf_reg': 0.4053689557345566, 'one_hot_max_size': 10, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 1 with value: 0.8657620717799731.
[I 2021-07-12 19:22:00,724] Trial 3 finished with value: 0.8665240467792362 and parameters: {'iterations': 300, 'learning_rate': 0.049205164888947

...

[I 2021-07-12 20:51:53,189] Trial 72 finished with value: 0.870344136914314 and parameters: {'iterations': 1200, 'learning_rate': 0.028387651750897042, 'random_strength': 1, 'bagging_temperature': 5, 'max_bin': 8, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 4, 'max_depth': 8, 'l2_leaf_reg': 5.6327160448886895, 'one_hot_max_size': 500, 'auto_class_weights': 'Balanced'}. Best is trial 57 with value: 0.8708849878265229.


Stopped by overfitting detector  (8 iterations wait)


[I 2021-07-12 20:52:43,942] Trial 73 finished with value: 0.8693384248935571 and parameters: {'iterations': 1200, 'learning_rate': 0.02091958137615044, 'random_strength': 1, 'bagging_temperature': 5, 'max_bin': 8, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 4, 'max_depth': 8, 'l2_leaf_reg': 0.7183954893316693, 'one_hot_max_size': 500, 'auto_class_weights': 'Balanced'}. Best is trial 57 with value: 0.8708849878265229.


Stopped by overfitting detector  (8 iterations wait)


[I 2021-07-12 20:53:30,550] Trial 74 finished with value: 0.8694136207219428 and parameters: {'iterations': 1200, 'learning_rate': 0.029893633727196826, 'random_strength': 2, 'bagging_temperature': 4, 'max_bin': 8, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 3, 'max_depth': 7, 'l2_leaf_reg': 3.132524331015214, 'one_hot_max_size': 500, 'auto_class_weights': 'Balanced'}. Best is trial 57 with value: 0.8708849878265229.


In [19]:
trial = study.best_trial
final_model = CatBoostClassifier(verbose=False,  cat_features=categorical_features_indices, 
                          **trial.params)

final_model.fit(X_train, y_train)

final_h1n1_model = final_model
trial.params

{'iterations': 1200,
 'learning_rate': 0.023793510396254353,
 'random_strength': 1,
 'bagging_temperature': 6,
 'max_bin': 8,
 'grow_policy': 'Lossguide',
 'min_data_in_leaf': 8,
 'max_depth': 9,
 'l2_leaf_reg': 89.35313522855303,
 'one_hot_max_size': 500,
 'auto_class_weights': 'Balanced'}

In [33]:
params = trial.params
final_h1n1_model = CatBoostClassifier(cat_features=categorical_features_indices,
                                     verbose=False,
                                     iterations=1200,
                                     learning_rate=0.023793510396254353,
                                     random_strength=1,
                                     bagging_temperature=6,
                                     max_bin=8,
                                     grow_policy='Lossguide',
                                     min_data_in_leaf=8,
                                     max_depth=9,
                                     l2_leaf_reg=89.35313522855303,
                                     one_hot_max_size=500,
                                     auto_class_weights='Balanced').fit(h1n1_train_trans, h1n1_labels)

In [21]:
params = trial.params
final_h1n1_model = CatBoostClassifier(cat_features=categorical_features_indices,
                                     verbose=False,
                                     **params)

# Seasonal

## Catboost and Optuna

In [22]:
X = seas_train_trans
y = seas_labels

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [24]:
from catboost import CatBoostClassifier
from catboost import Pool, cv
from sklearn.metrics import roc_curve, roc_auc_score
import optuna

In [25]:
train_dataset = Pool(data=X_train,
                    label=y_train,
                    cat_features=categorical_features_indices)

In [26]:
def objective(trial):
    param = {
        'iterations':trial.suggest_categorical('iterations', [100,200,300,500,1000,1200,1500,1700,2000]),
        'learning_rate':trial.suggest_float("learning_rate", 0.001, 0.3),
        'random_strength':trial.suggest_int("random_strength", 1,10),
        'bagging_temperature':trial.suggest_int("bagging_temperature", 0,10),
        'max_bin':trial.suggest_categorical('max_bin', [4,5,6,8,10,20,30]),
        'grow_policy':trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'min_data_in_leaf':trial.suggest_int("min_data_in_leaf", 1,10),
        'od_type' : "Iter",
        'od_wait' : 100,
        "depth": trial.suggest_int("max_depth", 2,10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
         'one_hot_max_size':trial.suggest_categorical('one_hot_max_size', [5,10,12,25,100,500,1024]),
        'custom_metric' : ['AUC'],
        "loss_function": "Logloss",
        'auto_class_weights':trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']),
        }

    scores = cv(train_dataset,
            param,
            fold_count=7, 
            early_stopping_rounds=8,         
            plot=False, verbose=False)

    return scores['test-AUC-mean'].max()

In [27]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=75)

[I 2021-07-12 20:53:53,796] A new study created in memory with name: no-name-d01ccdb2-195e-46e4-9ba1-c16b55870384


Stopped by overfitting detector  (8 iterations wait)


[I 2021-07-12 20:54:07,833] Trial 0 finished with value: 0.8580664207414844 and parameters: {'iterations': 200, 'learning_rate': 0.2127137007610176, 'random_strength': 1, 'bagging_temperature': 10, 'max_bin': 4, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 3, 'max_depth': 5, 'l2_leaf_reg': 0.0003636636071695854, 'one_hot_max_size': 5, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 0 with value: 0.8580664207414844.


Stopped by overfitting detector  (8 iterations wait)


[I 2021-07-12 20:54:15,446] Trial 1 finished with value: 0.858374522877302 and parameters: {'iterations': 100, 'learning_rate': 0.2728868002215558, 'random_strength': 3, 'bagging_temperature': 7, 'max_bin': 10, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 1, 'max_depth': 3, 'l2_leaf_reg': 2.833171661121252e-08, 'one_hot_max_size': 25, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 1 with value: 0.858374522877302.
[I 2021-07-12 20:55:12,541] Trial 2 finished with value: 0.8588582664911347 and parameters: {'iterations': 200, 'learning_rate': 0.023139350868493017, 'random_strength': 4, 'bagging_temperature': 1, 'max_bin': 4, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 2, 'max_depth': 8, 'l2_leaf_reg': 0.4053689557345566, 'one_hot_max_size': 10, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 2 with value: 0.8588582664911347.
[I 2021-07-12 20:56:21,743] Trial 3 finished with value: 0.8610563616945875 and parameters: {'iterations': 300, 'learning_rate': 0.04920516488894732

...

[I 2021-07-12 22:19:18,308] Trial 72 finished with value: 0.8631210524060586 and parameters: {'iterations': 2000, 'learning_rate': 0.048552385823553854, 'random_strength': 1, 'bagging_temperature': 0, 'max_bin': 20, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 4, 'max_depth': 5, 'l2_leaf_reg': 0.44088219820618896, 'one_hot_max_size': 10, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 68 with value: 0.8637094552038512.


Stopped by overfitting detector  (8 iterations wait)


[I 2021-07-12 22:20:12,117] Trial 73 finished with value: 0.8623630733076553 and parameters: {'iterations': 2000, 'learning_rate': 0.04820652592477086, 'random_strength': 1, 'bagging_temperature': 0, 'max_bin': 20, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 4, 'max_depth': 6, 'l2_leaf_reg': 0.3089687450075541, 'one_hot_max_size': 10, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 68 with value: 0.8637094552038512.


Stopped by overfitting detector  (8 iterations wait)


[I 2021-07-12 22:20:51,649] Trial 74 finished with value: 0.8624337826045686 and parameters: {'iterations': 2000, 'learning_rate': 0.08950725519297613, 'random_strength': 2, 'bagging_temperature': 0, 'max_bin': 20, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 3, 'max_depth': 5, 'l2_leaf_reg': 1.4122647681571165, 'one_hot_max_size': 10, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 68 with value: 0.8637094552038512.


In [28]:
trial = study.best_trial
final_model = CatBoostClassifier(verbose=False,  cat_features=categorical_features_indices, 
                          **trial.params)
trial.params

{'iterations': 2000,
 'learning_rate': 0.0647020282001146,
 'random_strength': 2,
 'bagging_temperature': 0,
 'max_bin': 30,
 'grow_policy': 'Depthwise',
 'min_data_in_leaf': 4,
 'max_depth': 5,
 'l2_leaf_reg': 50.04446925880975,
 'one_hot_max_size': 10,
 'auto_class_weights': 'SqrtBalanced'}

In [34]:
params = trial.params
final_seas_model = CatBoostClassifier(cat_features=categorical_features_indices,
                                     verbose=False,
                                     iterations=2000,
                                     learning_rate=0.0647020282001146,
                                     random_strength=2,
                                     bagging_temperature=0,
                                     max_bin=30,
                                     grow_policy='Depthwise',
                                     min_data_in_leaf=4,
                                     max_depth=5,
                                     l2_leaf_reg=50.04446925880975,
                                     one_hot_max_size=10,
                                     auto_class_weights='SqrtBalanced').fit(seas_train_trans, seas_labels)

In [30]:
params = trial.params
final_seas_model = CatBoostClassifier(cat_features=categorical_features_indices,
                                     verbose=False,
                                     **params)

### Train Accuracy

In [35]:
h1n1_train_data = train_test.copy()
h1n1_train_data[num_cols] = h1n1_scaler.transform(h1n1_train_data[num_cols])

seas_train_data = train_test.copy()
seas_train_data[num_cols] = seas_scaler.transform(seas_train_data[num_cols])

In [36]:
y_predicted_h1n1 = final_h1n1_model.predict_proba(h1n1_train_data)[:,1].reshape(-1,1)
y_predicted_seas = final_seas_model.predict_proba(seas_train_data)[:,1].reshape(-1,1)

y_true = np.array(labels)

y_predicted = np.concatenate((y_predicted_h1n1, y_predicted_seas), axis=1)

In [37]:
roc_auc_score(np.array(labels), y_predicted)

0.9265800144791575

# Submission

In [38]:
test = pd.read_csv('../Data/test_set_features.csv', index_col='respondent_id')
full_test = test.copy()

In [39]:
num_cols = list(test.select_dtypes('number').columns)

cat_cols = [
    'race',
    'sex',
    'marital_status',
    'rent_or_own',
    'hhs_geo_region',
    'census_msa',
    'employment_industry',
    'employment_occupation'
]

ord_cols = [
    'age_group',
    'education',
    'income_poverty',
    'employment_status'
]


#Impute Test
for col in num_cols:
    test[col] = test[col].fillna(value=-1)


for col in (cat_cols+ord_cols):
    test[col] = test[col].fillna(value='None')

    
test['age_group'] = test['age_group'].map({
    '18 - 34 Years': 1,
    '35 - 44 Years': 2,
    '45 - 54 Years': 3,
    '55 - 64 Years': 4,
    '65+ Years': 5
})
    
test['education'] = test['education'].map({
    '< 12 Years': 1,
    '12 Years': 2,
    'Some College': 3,
    'College Graduate': 4,
    'None': -1
})

test['income_poverty'] = test['income_poverty'].map({
    'None': -1,
    'Below Poverty': 1,
    '<= $75,000, Above Poverty': 2,
    '> $75,000': 3
})

test['employment_status'] = test['employment_status'].map({
    'None': -1,
    'Unemployed': 1,
    'Employed': 2,
    'Not in Labor Force': 3
})

In [40]:
test_h1n1 = test.copy()
test_seas = test.copy()

test_h1n1[num_cols] = h1n1_scaler.transform(test_h1n1[num_cols])
test_seas[num_cols] = seas_scaler.transform(test_seas[num_cols])

In [41]:
y_h1n1 = final_h1n1_model.predict_proba(test_h1n1)[:,1].reshape(-1,1)
y_seas = final_seas_model.predict_proba(test_seas)[:,1].reshape(-1,1)

y_comb = np.concatenate((y_h1n1, y_seas), axis=1)

In [42]:
y_comb

array([[0.28747632, 0.15994814],
       [0.11570729, 0.01653142],
       [0.41235066, 0.79593744],
       ...,
       [0.38341579, 0.13381159],
       [0.05137968, 0.34525411],
       [0.81370657, 0.58536735]])

In [43]:
results = pd.DataFrame(y_comb, columns=['h1n1_vaccine', 'seasonal_vaccine'], index=test.index)

submission = pd.concat([full_test, results], axis=1)
submission = submission[['h1n1_vaccine', 'seasonal_vaccine']]

In [44]:
submission

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.287476,0.159948
26708,0.115707,0.016531
26709,0.412351,0.795937
26710,0.868239,0.903654
26711,0.616027,0.497508
...,...,...
53410,0.618118,0.489700
53411,0.405331,0.300675
53412,0.383416,0.133812


In [45]:
today = datetime.today().date()

submission.to_csv(f'../Submissions/Neural Network Submission {today}.csv')

In [ ]:
submission